In [1]:
!date
import pandas as pd
import numpy as np
import sys
sys.path.append('../GOCAM_Project/dev')

import rpy2
from rpy2.robjects.packages import importr
BiasedUrn = importr('BiasedUrn')

import utils
import enrich
import numba

pd.options.display.max_colwidth = 100

Wed Dec 14 17:23:04 PST 2022


# Deprecated Code

The following functions have been deprecated because the runtime was bad (although the conversion to binary was super fast and pretty cool). I'm leaving them here, unused, because they can provide validation.

Originally, all possible inclusions/exclusions the sets were enumerated into a matrix of size $2^{|sets|}$, and then the number of single-protein entries was appended to the start of each vector, with that number ranging from k to the size of the gocam. It essentially enumerated all successes in the sample space, where each set is treated individually. 

The newer set of functions groups these sets by their sizes and then enumerates all successes, but it vastly reduces the number of calculations by eliminating redundant calculations where the sum of the number of entities across all sets of a particular size doesn't change because the "count" is redistributed within that group.

In [2]:
def unpackbits_DEPRECATED(x, num_bits):
    if np.issubdtype(x.dtype, np.floating):
        raise ValueError("numpy data type needs to be int-like")
    xshape = list(x.shape)
    x = x.reshape([-1, 1])
    mask = 2**np.arange(num_bits, dtype=x.dtype).reshape([1, num_bits])
    return (x & mask).astype(bool).astype(int).reshape(xshape + [num_bits])

In [3]:
#https://stackoverflow.com/questions/18296035/how-to-extract-the-bits-of-larger-numeric-numpy-data-types
def make_XT_DEPRECATED(k,N,m1,y):
    l = np.power(2,y)
    XT = []
    for k_ in np.arange(k,y+m1+1): #+1 because it doesn't include the last number
        x1_min = max(0, k_ - y) # if no sets are utilized, all k_ balls have to be in the m1 bin
        x1_max = min(k_,m1)

        a = np.array(range(l), dtype=np.uint16)
        Xt = unpackbits(a, num_bits = y)
        Xt = Xt[:,0:y]
        x1_vec = k_- np.sum(Xt, axis = 1) #how many balls are left over after the bins for sets are utilized

       
        Xt = np.concatenate((x1_vec.reshape(len(x1_vec),1),Xt), axis = 1)
        mask1 = Xt[:,0] <= x1_max
        Xt = Xt[mask1]
        mask2 = Xt[:,0] >= x1_min
        Xt = Xt[mask2]
        x_mp1_vec = N- np.sum(Xt, axis = 1) #number of balls to be drawn from the (m+1)th bin (the non-gocam background)
        Xt = np.concatenate((Xt,x_mp1_vec.reshape(len(x_mp1_vec),1)), axis = 1)
        XT.append(Xt)
    XT = np.concatenate(XT, axis = 0)
    np.save('../temp/XT.npy',XT)
    
    return XT

In [4]:
def do_ncHGT_DEPRECATED(k,gc):
    setID2members = utils.csv2dict('../data/setID2members.csv')
    gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')
   
    M = 5300
    N = 766-496


    w_gc = [1] #initialize with 1 as the weight of single proteins (irrespective of whether there are any)
    m_gc = [0] #initialize with 0 single proteins
    num_protein = 0
    for i in gocam2ID.get(gc):
        if "sset:" in i:
            w_i = len(setID2members.get(i))
            w_gc.append(w_i)
            m_gc.append(1)
        else:
            num_protein+=1
    m_gc[0] = num_protein
    m_gc.append(M-np.sum(m_gc)) #entities not in the gocam (roughly)
    w_gc.append(1) #weight for entities not in the gocam (all weighted as 1)

    y = len(m_gc)-2 #first and last positions are not sets. they are single proteins and the rest of the Dict, respectively

    XT = make_XT(k,N,m_gc[0],y)

    m = rpy2.robjects.IntVector(m_gc)
    w = rpy2.robjects.IntVector(w_gc)
    pval = ncHGT_sf(XT,m,N,w)
    return XT,m_gc,w_gc

# Setup

In [5]:
gene_list = pd.read_csv('../../Desktop/GOCAM/P97.csv',header=None,names = ['g'])
P97_test_set = gene_list.g
uniprot2input = pd.Series(P97_test_set.values,index=P97_test_set).to_dict()
x = pd.read_csv('../data/gocam_sizes_mouse.csv')
gocam_sizes = pd.Series(x.sizes.values,index=x.gocam)
Dict = utils.csv2dict('../data/ID2gocam_mouse.csv')
filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display= enrich.enrich(list(gene_list.g), P97_test_set, uniprot2input, gocam_sizes, Dict,alpha = .05, show_significant = False)
#df_display['pval ncHGT'] = p_f
df_display = df_display[df_display.title.apply(lambda x: 'APC/C:Cdh1 mediated' in x)]


In [6]:
gene_list = pd.read_csv('../../Desktop/GOCAM/cancer.csv',header=None,names = ['g'])
cancer_test_set, uniprot2input, not_converted = utils.convert_IDs(gene_list,'Gene Symbol')
x = pd.read_csv('../data/gocam_sizes_mouse.csv')
gocam_sizes = pd.Series(x.sizes.values,index=x.gocam)
Dict = utils.csv2dict('../data/ID2gocam_mouse.csv')
filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display= enrich.enrich(list(gene_list.g), cancer_test_set, uniprot2input, gocam_sizes, Dict, alpha = .05)
df_display.head()

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
0,Role of LAT2/NTAL/LAB on calcium mobilization - Reactome,1.590968e-07,6,7,"[P42336, P43405, sset:SYK/p-LYN, sset:PI3K-regulatory subunit, sset:PI3K-catalytic subunit, sset...",http://model.geneontology.org/R-HSA-2730905
1,CD28 dependent PI3K/Akt signaling - Reactome,5.812643e-07,8,16,"[P42336, P27986, sset:Activator:PI3K, sset:Src family tyrosine kinases (SFKs), sset:PI3K alpha, ...",http://model.geneontology.org/R-HSA-389357
2,DAP12 signaling - Reactome,6.076063e-07,6,8,"[P43405, sset:FYN,LCK, sset:PI3K-regulatory subunit, sset:PI3K-catalytic subunit, sset:p-SYK,p-B...",http://model.geneontology.org/R-HSA-2424491
3,Stabilization of p53 - Reactome,6.076063e-07,6,8,"[Q13315, O96017, Q00987, O15151, P04637, sset:p-S166,S188-MDM2 dimer, p-S166,S188-MDM2:MDM4]",http://model.geneontology.org/R-HSA-69541
4,Downstream signal transduction - Reactome,1.740445e-06,6,9,"[P01127, P16234, P09619, P42336, sset:Phospho- PDGF receptor dimer, sset:Active PDGF dimers (AA,...",http://model.geneontology.org/R-HSA-186763


In [7]:
p_f = [3.8E-7,1.3E-5,3.6E-7,1.3E-7,4.4E-5,.00022,1.26E-5,3.1E-5,2.85E-5,.0014,.0013,.00035,'NA',2.04E-5,4.13E-6,.0004,.0011,5.94E-5,'NA','NA',9.7E-5]

# New Code
(still needs to be verified. See the note under deprecation for an explanation of this method)

In [8]:
def ncHGT_sf(XT,m,N,w):
    #l = len(XT)/len(m)
    pval = 0
    for i in range(len(XT)):
        x = rpy2.robjects.IntVector(XT[i])
        pval = pval + BiasedUrn.dMFNCHypergeo(x,m,N,w)[0]
    return pval

In [9]:
#enumerate all possible counts in the bins entities of each weight (weighted by set size)
    #excluding the non-gocam bin, which will be added later
def f(m_new,i,prev_array):
    first = True
    for j in range(m_new[i]+1):
        xt = prev_array.copy()
        xt[0][i] = j
        
        #recursion
        if (i < len(m_new)-1):
            xt = f(m_new, i+1, xt) #will return array of arrays
            
        #combining results
        if not first:
            XT = np.concatenate([XT,xt], axis = 0)
        else:
            XT = xt
            first = False
    return XT

In [10]:
def make_new_vectors(w_gc,m_gc):
    w_temp = np.sort(w_gc[:-1])
    prev = 1 #first entry?
    m_temp=[m_gc[0]]
    for x in w_temp:
        if x == prev:
            m_temp[-1] += 1
        else:
            prev = x
            m_temp.append(1)
    w_new = np.append(np.unique(w_gc),np.array([1]))#np.sort returns unique values in sorted order ("compresses" w_temp) and we append 1 for the weight of non-gocam entities
    return w_new, m_temp

In [11]:
def make_initial_vectors(gocam2ID,setID2members, gc, M):
    w_gc = [1] #initialize with 1 as the weight of single proteins (irrespective of whether there are any)
    m_gc = [0] #initialize with 0 single proteins
    num_protein = 0
    for i in gocam2ID.get(gc):
        if "sset:" in i:
            w_i = len(setID2members.get(i))
            w_gc.append(w_i)
            m_gc.append(1)
        else:
            num_protein+=1
    m_gc[0] = num_protein
    m_gc.append(M-np.sum(m_gc)) #entities not in the gocam (roughly)
    w_gc.append(1) #weight for entities not in the gocam (all weighted as 1)
    return w_gc, m_gc

In [12]:
def do_ncHGT_new(k,gc):
    setID2members = utils.csv2dict('../data/setID2members.csv')
    gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')

    M = 5300
    N = 766-496
    
    #make weight (w) and bin size (m) vectors where each entity in the gocam gets its own entry
    w_in, m_in = make_initial_vectors(gocam2ID, setID2members, gc, M)
    
    #update m and w vectors by grouping sets of the same size
    w_new , m_new= make_new_vectors(w_in,m_in)

    #make XT matrix, an enumeration of all possible arangements of balls in bins based on m_new and w_new
    XT = f(m_new,0,np.zeros(shape=(1,len(m_new))))
    m_new.append(M-np.sum(m_new)) #add the last bin, corresponding to entities not in the gocam (roughly)
    
    #filter XT to only include the region of the sample space >= k (which is what we want to sum probabilities over)
    mask1 = (np.sum(XT, axis=1) >= k)
    XT = XT[mask1]

    #filter XT to ensure that more than N entities are not picked
    mask2 = (np.sum(XT, axis=1) <= N)
    XT = XT[mask2]

    #add the remaining entities to the m+1th bin (non gocam bin)
    x_mp1_vec = N- np.sum(XT, axis = 1) #number of balls to be drawn from the last bin (the non-gocam background)
    XT = np.concatenate((XT,x_mp1_vec.reshape(len(x_mp1_vec),1)), axis = 1)

    m = rpy2.robjects.IntVector(m_new)
    w = rpy2.robjects.IntVector(w_new)
    pval = ncHGT_sf(XT,m,N,w)
    return pval


In [13]:
df_display['ncHGT'] = df_display.apply(lambda x: do_ncHGT_new( x['# genes in list'], x['url']), axis=1)


In [14]:
%%timeit -n 1 -r 1
df_display.apply(lambda x: do_ncHGT_new( x['# genes in list'], x['url']), axis=1) 

59.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
df_display

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url,ncHGT
0,Role of LAT2/NTAL/LAB on calcium mobilization - Reactome,1.590968e-07,6,7,"[P42336, P43405, sset:SYK/p-LYN, sset:PI3K-regulatory subunit, sset:PI3K-catalytic subunit, sset...",http://model.geneontology.org/R-HSA-2730905,0.000022
1,CD28 dependent PI3K/Akt signaling - Reactome,5.812643e-07,8,16,"[P42336, P27986, sset:Activator:PI3K, sset:Src family tyrosine kinases (SFKs), sset:PI3K alpha, ...",http://model.geneontology.org/R-HSA-389357,0.000673
2,DAP12 signaling - Reactome,6.076063e-07,6,8,"[P43405, sset:FYN,LCK, sset:PI3K-regulatory subunit, sset:PI3K-catalytic subunit, sset:p-SYK,p-B...",http://model.geneontology.org/R-HSA-2424491,0.000022
3,Stabilization of p53 - Reactome,6.076063e-07,6,8,"[Q13315, O96017, Q00987, O15151, P04637, sset:p-S166,S188-MDM2 dimer, p-S166,S188-MDM2:MDM4]",http://model.geneontology.org/R-HSA-69541,0.000001
4,Downstream signal transduction - Reactome,1.740445e-06,6,9,"[P01127, P16234, P09619, P42336, sset:Phospho- PDGF receptor dimer, sset:Active PDGF dimers (AA,...",http://model.geneontology.org/R-HSA-186763,0.000009
5,VEGFR2 mediated vascular permeability - Reactome,4.705982e-06,8,20,"[P31749, P42336, P27986, P07900, sset:Activator:PI3K, sset:PI3K alpha, beta, gamma, sset:PI3K-re...",http://model.geneontology.org/R-HSA-5218920,0.001879
6,Negative regulation of the PI3K/AKT network - Reactome,8.610631e-06,5,7,"[P60484, sset:Activator:PI3K, sset:PIK3C(1), sset:PI3K-regulatory subunit, sset:PI3K-catalytic s...",http://model.geneontology.org/R-HSA-199418,0.003187
7,GPVI-mediated activation cascade - Reactome,1.086604e-05,8,22,"[P42336, P27986, P43405, sset:Activator:PI3K, sset:p-Y348-SYK/LCK, sset:PI3K alpha, beta, gamma,...",http://model.geneontology.org/R-HSA-114604,0.001395
8,PI3K events in ERBB2 signaling - Reactome,2.956176e-05,6,13,"[P00533, P04626, P42336, P27986, sset:ERBB2 heterodimers, sset:p-Y877-ERBB2 heterodimers]",http://model.geneontology.org/R-HSA-1963642,0.002299
9,Oncogene Induced Senescence - Reactome,4.032287e-05,4,5,"[Q00987, P04637, sset:p-T172-CDK4,p-T177-CDK6:CCND:CDKN1A,CDKN1B,(CDKN1C); p-T172-CDK4,p-T177-CD...",http://model.geneontology.org/R-HSA-2559585,0.000588


# Examining whether sets tend to come from multiple cell types or really represent just one

In [16]:
import random

In [17]:
setID2members = utils.csv2dict('../data/setID2members.csv')
random.seed(a=1)
sets = random.choices(list(setID2members.items()), k =20)

In [18]:
#sets

# Trying ncHGT on the data directly

P97 with standard method

In [51]:
df_s = enrich.enrich_wrapper('P97.csv','uniprot',method='set',show_significant=False)
np.sum(df_s['pval (uncorrected)'].values < 5e-03)

25

In [65]:
df_s[df_s.title.apply(lambda x: 'RHOA GTPase cycle - Reactome' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
33,RHOA GTPase cycle - Reactome,0.007182,2,2,"[sset:RHOA GAPs, sset:RHOA GEFs]",http://model.geneontology.org/R-HSA-8980692


In [55]:
df_ff = enrich.enrich_wrapper('P97.csv','uniprot',method='standard',show_significant=False)
np.sum(df_ff['pval (uncorrected)'].values < 5e-04)

40

In [57]:
df_ff['ncHGT'] = df_ff.apply(lambda x: do_ncHGT_new( x['# genes in list'], x['url']), axis=1)
df_ff

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url,ncHGT
0,Unwinding of DNA - Reactome,1.009108e-07,6,7,"[P25205, Q14566, P33991, P49736, P33993, P33992]",http://model.geneontology.org/R-HSA-176974,4.252979e-07
1,SCF(Skp2)-mediated degradation of p27/p21 - Reactome,6.327843e-07,13,50,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, P24941, P38936, Q92530, P46527,...",http://model.geneontology.org/R-HSA-187577,9.119390e-07
2,Autodegradation of Cdh1 by Cdh1:APC/C - Reactome,4.803101e-06,13,59,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,...",http://model.geneontology.org/R-HSA-174084,6.975239e-06
3,CDK-mediated phosphorylation and removal of Cdc6 - Reactome,4.844141e-06,14,68,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, P24941, Q92530, Q16763, O00762,...",http://model.geneontology.org/R-HSA-69017,5.731080e-06
4,Assembly of the pre-replicative complex - Reactome,5.860558e-06,13,60,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,...",http://model.geneontology.org/R-HSA-68867,8.466233e-06
...,...,...,...,...,...,...,...
475,Regulation of Complement cascade - Reactome,9.977228e-01,1,117,[P15529],http://model.geneontology.org/R-HSA-977606,9.571815e-01
476,Antigen activates B Cell Receptor (BCR) leading to generation of second messengers - Reactome,9.994799e-01,1,145,[P16333],http://model.geneontology.org/R-HSA-983695,9.949331e-01
477,G alpha (12/13) signalling events - Reactome,9.999092e-01,3,268,"[Q92974, P07602, Q9HCE6]",http://model.geneontology.org/R-HSA-416482,6.875309e-01
478,G alpha (i) signalling events - Reactome,9.999717e-01,2,250,"[P07602, P49795]",http://model.geneontology.org/R-HSA-418594,8.972519e-01


In [59]:
background_num_gocams = 1100
alpha = .05
test = df_ff.copy()
test.sort_values('pval (uncorrected)',inplace=True)
test.reset_index(drop=True, inplace=True)
test['FDR_val'] = (test.index+1)*alpha/background_num_gocams
test['Less_than'] = (test['pval (uncorrected)'] < test['FDR_val'])
index = test.Less_than.where(test.Less_than==True).last_valid_index()
index

63

In [60]:
background_num_gocams = 1100
alpha = .05
test = df_ff.copy()
test.sort_values('ncHGT',inplace=True)
test.reset_index(drop=True, inplace=True)
test['FDR_val'] = (test.index+1)*alpha/background_num_gocams
test['Less_than'] = (test['ncHGT'] < test['FDR_val'])
index = test.Less_than.where(test.Less_than==True).last_valid_index()
index

67

In [61]:
.05*67/1100

0.0030454545454545456

In [62]:
df_ff[df_ff.title.apply(lambda x: 'PI5P' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url,ncHGT
198,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.218219,8,115,"[P00533, P30154, P62714, Q9BTU6, Q99755, P35568, Q8NCE2, O14944]",http://model.geneontology.org/R-HSA-6811558,0.00025


In [63]:
test[0:67]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url,ncHGT,FDR_val,Less_than
0,RHOA GTPase cycle - Reactome,6.418819e-01,4,87,"[Q52LW3, Q92974, Q9H0H5, Q9HCE6]",http://model.geneontology.org/R-HSA-8980692,0.000000e+00,0.000045,True
1,RAC1 GTPase cycle - Reactome,7.466143e-01,4,100,"[Q52LW3, Q8IWW6, Q9H0H5, Q9BZ29]",http://model.geneontology.org/R-HSA-9013149,0.000000e+00,0.000091,True
2,Antigen processing: Ubiquitination & Proteasome degradation - Reactome,1.739141e-05,33,305,"[Q7Z6Z7, Q9ULT8, Q13200, Q14669, P43686, P62195, P62191, O00231, P35998, Q9UNM6, Q05086, O00487,...",http://model.geneontology.org/R-HSA-983168,3.096914e-28,0.000136,True
3,RAF activation - Reactome,3.024910e-01,9,145,"[P00533, P30154, P62714, Q96S59, O15020, Q13555, Q92796, P35568, O14944]",http://model.geneontology.org/R-HSA-5673000,4.263451e-08,0.000182,True
4,Unwinding of DNA - Reactome,1.009108e-07,6,7,"[P25205, Q14566, P33991, P49736, P33993, P33992]",http://model.geneontology.org/R-HSA-176974,4.252979e-07,0.000227,True
...,...,...,...,...,...,...,...,...,...
62,"Antigen Presentation: Folding, assembly and peptide loading of class I MHC - Reactome",1.273079e-03,9,54,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530]",http://model.geneontology.org/R-HSA-983170,1.396877e-03,0.002864,True
63,Neddylation - Reactome,1.161070e-03,10,64,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q9UBF6]",http://model.geneontology.org/R-HSA-8951664,1.416228e-03,0.002909,True
64,GLI3 is processed to GLI3R by the proteasome - Reactome,1.273079e-03,9,54,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530]",http://model.geneontology.org/R-HSA-5610785,1.520465e-03,0.002955,True
65,Degradation of GLI2 by the proteasome - Reactome,1.273079e-03,9,54,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530]",http://model.geneontology.org/R-HSA-5610783,1.520465e-03,0.003000,True
